In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

We Will be working on the heart disease dataset

In [ ]:
#loading the csv data to a pandas dataframe
heart_data = pd.read_csv('/content/heart_disease_data.csv')

In [ ]:
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
heart_data.shape

(303, 14)

In [ ]:
heart_data.isnull().sum()

,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


In [ ]:
heart_data['target'].value_counts()

,count
target,
1,165
0,138


1-->Defective Heart
0-->Healthy Heart

Splitting the Features and Target

In [ ]:
x=heart_data.drop(columns='target',axis=1)
y=heart_data['target']

In [ ]:
print(x)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  
0        0   0     1  
1        0   0     2  
2        2   0    

In [ ]:
print(y)

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64


In [ ]:
x=np.asarray(x)
y=np.asarray(y)

Model Selection

1. Comparing the models with default hyperpara values using cross validation

In [ ]:
#list of models
models=[LogisticRegression(max_iter=1000),SVC(kernel='linear'),KNeighborsClassifier(),RandomForestClassifier(random_state=0)]

In [ ]:
def compare_models_cross_validation():
  for model in models:
    cv_score=cross_val_score(model,x,y,cv=5)
    mean_accuracy=sum(cv_score)/len(cv_score)
    mean_accuracy=mean_accuracy*100
    mean_accuracy=round(mean_accuracy,2)
    print('Cross validation accuracies for the',model,'=',cv_score)
    print('Accuracy score of the ',model,'=',mean_accuracy,'%')
    print('------------------------------------------------')


In [ ]:
compare_models_cross_validation()

Cross validation accuracies for the LogisticRegression(max_iter=1000) = [0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
Accuracy score of the  LogisticRegression(max_iter=1000) = 82.83 %
------------------------------------------------
Cross validation accuracies for the SVC(kernel='linear') = [0.81967213 0.8852459  0.80327869 0.86666667 0.76666667]
Accuracy score of the  SVC(kernel='linear') = 82.83 %
------------------------------------------------
Cross validation accuracies for the KNeighborsClassifier() = [0.60655738 0.6557377  0.57377049 0.73333333 0.65      ]
Accuracy score of the  KNeighborsClassifier() = 64.39 %
------------------------------------------------
Cross validation accuracies for the RandomForestClassifier(random_state=0) = [0.85245902 0.90163934 0.81967213 0.81666667 0.8       ]
Accuracy score of the  RandomForestClassifier(random_state=0) = 83.81 %
------------------------------------------------


Inference : For the heart disease dataset, **random forest** has the highest accuracy value with default hyperpara values

2.Comparing the models with different hyperpara values using gridsearchcv

In [26]:
models_list=[LogisticRegression(max_iter=10000),SVC(),KNeighborsClassifier(),RandomForestClassifier(random_state=0)]

In [ ]:
#creating a dict that has hyperpara values

model_hyperpara={
    'log_reg_para':{'C':[1,5,10,20]},
    'svc_para':{'kernel':['linear','poly','rbf','sigmoid'],'C':[1,5,10,20]},
    'KNN_para':{'n_neighbors':[3,5,10]},
    'random_forest_para':{'n_estimators':[10,20,50,100]}}

In [ ]:
type(model_hyperpara)

dict

In [ ]:
dict_keys=model_hyperpara.keys()
print(dict_keys)

dict_keys(['log_reg_para', 'svc_para', 'KNN_para', 'random_forest_para'])


In [ ]:
model_keys=list(dict_keys)
print(model_keys)

['log_reg_para', 'svc_para', 'KNN_para', 'random_forest_para']


In [ ]:
model_hyperpara['svc_para']

{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}

In [22]:
model_keys[2]

'KNN_para'

In [24]:
model_hyperpara[model_keys[1]]

{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}

Applying GridSearchCV

In [28]:
def ModelSelection(list_of_models,hyperpara_dict):
  result=[]
  i=0
  for model in list_of_models:
    key=model_keys[i]
    params=hyperpara_dict[key]
    i+=1
    print(model)
    print(params)
    print('------------------------------------------------')
    classifier=GridSearchCV(model,params,cv=5)
    classifier.fit(x,y)
    result.append({
        'model used':model,
        'highest score':classifier.best_score_,
        'best hyperpara':classifier.best_params_
    })
  result_dataframe=pd.DataFrame(result,columns=['model used','highest score','best hyperpara'])
  return result_dataframe



In [29]:
ModelSelection(models_list,model_hyperpara)

LogisticRegression(max_iter=10000)
{'C': [1, 5, 10, 20]}
------------------------------------------------
SVC()
{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}
------------------------------------------------
KNeighborsClassifier()
{'n_neighbors': [3, 5, 10]}
------------------------------------------------
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50, 100]}
------------------------------------------------


,model used,highest score,best hyperpara
0,LogisticRegression(max_iter=10000),0.831585,{'C': 5}
1,SVC(),0.828306,"{'C': 1, 'kernel': 'linear'}"
2,KNeighborsClassifier(),0.643880,{'n_neighbors': 5}
3,RandomForestClassifier(random_state=0),0.838087,{'n_estimators': 100}


RandomForestClassifier with n_estimators=100 is the best here